In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("data/PS_20174392719_1491204439457_log.csv")
data.head(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
data["nameOrig"].unique()

array(['C1231006815', 'C1666544295', 'C1305486145', ..., 'C1162922333',
       'C1685995037', 'C1280323807'], shape=(6353307,), dtype=object)

In [7]:
data["nameOrig"].value_counts()

nameOrig
C1677795071    3
C1999539787    3
C724452879     3
C1976208114    3
C400299098     3
              ..
C1970706589    1
C40604503      1
C1614818636    1
C2089752665    1
C154988899     1
Name: count, Length: 6353307, dtype: int64

In [8]:
data.shape

(6362620, 11)

In [9]:
data.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [10]:
data.duplicated().sum()

np.int64(0)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [12]:
data["hour"] = pd.to_timedelta(data["step"] , unit = "h")

In [13]:
data["hour"].head()

0   0 days 01:00:00
1   0 days 01:00:00
2   0 days 01:00:00
3   0 days 01:00:00
4   0 days 01:00:00
Name: hour, dtype: timedelta64[ns]

In [14]:
base_time = pd.to_datetime("2017-01-01")

In [15]:
data["hour"] = base_time + data["hour"]
data["hour"].head()

0   2017-01-01 01:00:00
1   2017-01-01 01:00:00
2   2017-01-01 01:00:00
3   2017-01-01 01:00:00
4   2017-01-01 01:00:00
Name: hour, dtype: datetime64[ns]

In [16]:
data["hour"]=pd.to_datetime(data["hour"])

In [31]:
hourly_counts = data.groupby(['nameOrig', 'hour']).size().reset_index(name='transactions_count')
hourly_counts.shape

(6362573, 3)

In [32]:
hourly_counts.head()

,nameOrig,hour,transactions_count
0,C1000000639,2017-01-11 09:00:00,1
1,C1000001337,2017-01-10 01:00:00,1
2,C1000001725,2017-01-02 22:00:00,1
3,C1000002591,2017-01-10 15:00:00,1
4,C1000003372,2017-01-07 23:00:00,1


In [33]:
data["frequency/hour"] = data.set_index(["nameOrig","hour"]).index.map(
    hourly_counts.set_index(["nameOrig","hour"])["transactions_count"]
    )
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,hour,frequency/hour
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,2017-01-01 01:00:00,1
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,2017-01-01 01:00:00,1
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,2017-01-01 01:00:00,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,2017-01-01 01:00:00,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,2017-01-01 01:00:00,1


In [34]:
data["day"] = data["hour"].dt.date
data["day"].head()

0    2017-01-01
1    2017-01-01
2    2017-01-01
3    2017-01-01
4    2017-01-01
Name: day, dtype: object

In [35]:
daily_counts = data.groupby(["nameOrig" , "day"]).size().reset_index(name="day_counts")
daily_counts.head()

,nameOrig,day,day_counts
0,C1000000639,2017-01-11,1
1,C1000001337,2017-01-10,1
2,C1000001725,2017-01-02,1
3,C1000002591,2017-01-10,1
4,C1000003372,2017-01-07,1


In [36]:
data["frequency_day"] = data.set_index(["nameOrig","day"]).index.map(
    daily_counts.set_index(["nameOrig","day"])["day_counts"]
)

In [38]:
data["frequency_day"].head()

0    1
1    1
2    1
3    1
4    1
Name: frequency_day, dtype: int64